# Introduction to filtering an alert stream

For the hackathon closing out ZTF Summer School 2024, your challenge is to create your own alert broker which can consume a stream of alerts and filter for interesting objects.

This is a starter notebook for the hackathon. It will show you how to read a simulated ZTF alert stream and give some examples of how you can build your own filtering for those alerts. Previous notebooks from this week, for example "ztf_alert_filtering" from Day 1, provide additional examples that you may find useful to apply to this challenge.

In [ ]:
# install packages (feel free to add any you want for the hackathon):
!pip install astropy numpy pandas matplotlib pathlib fastavro io

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Part 1 - Reading the alert stream

Note: here we simulate our Kafka consumer by reading the data with an iterator function. Some people will find that their computers can run the true kafka consumer on these couple hundred thousand alerts on the order of minutes, but we use the iterator to ensure feasible timescales for all users.

In [ ]:
# TO ADD: count how many files are in data/20240707, ie how many alerts there were on 2024/07/07 that you will be processing in this challenge



In [6]:
# run this cell to create the iterator
import pathlib

def get_iterator(date):
    path = pathlib.Path("data") / date

    # grab the full list of alert files and create an iterator
    alert_files = sorted(list(path.glob("*.avro")))

    for file in alert_files:
        content = open(file, mode='rb').read()
        yield content

iterator = get_iterator("20240707")

In [7]:
# print an alert from the iterator
next(iterator)

b'Obj\x01\x04\x14avro.codec\x08null\x16avro.schema\xe0\x8f\x03{"doc":"avro alert schema for ZTF (www.ztf.caltech.edu)","fields":[{"doc":"schema version used","name":"schemavsn","type":"string"},{"doc":"origin of alert packet","name":"publisher","type":"string"},{"doc":"object identifier or name","name":"objectId","type":"string"},{"name":"candid","type":"long"},{"name":"candidate","type":{"doc":"avro alert schema","fields":[{"doc":"Observation Julian date at start of exposure [days]","name":"jd","type":"double"},{"doc":"Filter ID (1=g; 2=R; 3=i)","name":"fid","type":"int"},{"doc":"Processing ID for science image to facilitate archive retrieval","name":"pid","type":"long"},{"default":null,"doc":"Expected 5-sigma mag limit in difference image based on global noise estimate [mag]","name":"diffmaglim","type":["null","float"]},{"default":null,"doc":"filename of positive (sci minus ref) difference image","name":"pdiffimfilename","type":["null","string"]},{"default":null,"doc":"Principal inve

In [ ]:
import io
from fastavro import reader

#TO ADD: What format are the alerts given in? Convert the alerts to something more readable like dictionaries.

for alert in iterator:
    ...

In [ ]:
#TO ADD: Get the id for one of the alerts, and look it up on your choice of ZTF alert broker, for example alerce (https://alerce.online)
# you would be very lucky to find something interesting by chance, but this will be a helpful reference once you have designed a filter



# Part 2 - Filtering the alert stream

The next cells will start with some general suggested steps in building your filter. Some suggested techniques for filtering are:

- Make cuts on [alert parameters](https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html) (as you did in ztf_summer_school_2024/lectures/01/ztf_alert_filtering.ipynb)

- Use prv_candidates to understand the alert history, and do things like compute the rate that transients are rising and falling (hint: np.polyfit)

- Use a classifier trained in a previous notebook or train a new one

- Try something new !

In [ ]:
# To Add: it is recommended to start by looking at all of the fields in the alert schema, and look up things you don't undertand in the alert parameters
# link above.


In [ ]:
# To Add: As you did on the first day, one important first step in your filter will be to filter out "bogus" alerts. 
# Science groups actually have a wide range of ways to do this, from a simple cut on drb to much longer filters that use multiple different fields.
# Either (a) make a decision on what value to cut on drb at, or (b) design your own set of conditions to removes bogus alert



In [ ]:
# To Add: Another good first step of your filter would be to filter out solar system objects. 
# Hint: try using the fields ssdistnr and magnr to create such a filter. How many objects are you cutting out with this filter?



# Time to start!

Reminder: your challenge is to build filters to find your choice of transient with minimal time to run. Supernovae are good objects to try out, but you are welcome to experiment with any objects of your choice.

1. Supernovae
2. Object of your choice